In [1]:
from genai_stack.model.base import BaseModel, BaseModelConfig, BaseModelConfigModel
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from pydantic import Field

OSError: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /home/codespace/.python/current/lib/python3.10/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libllmodel.so)

In [2]:
class HuggingFaceModelConfigModel(BaseModelConfigModel):
    model_name: str = "meta-llama/Llama-2-70b-chat-hf"
    # You can use Field() from pydantic to add any other configuration options you need here.

In [3]:
class HuggingFaceModelConfig(BaseModelConfig):
    data_model = HuggingFaceModelConfigModel

In [4]:
class HuggingFaceModel(BaseModel):
    config_class = HuggingFaceModelConfig

    @classmethod
    def from_kwargs(cls, kwargs):
        model_name = kwargs.get("model_name")
        return cls(config=HuggingFaceModelConfig(data=HuggingFaceModelConfigModel(model_name=model_name)))

    def load(self):
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.model_name)
        self.model = AutoModelForCausalLM.from_pretrained(self.config.model_name)

    def predict(self, prompt: str):
        input_ids = self.tokenizer.encode(prompt, return_tensors="pt")
        output = self.model.generate(input_ids, max_length=100, num_return_sequences=1)
        response = self.tokenizer.decode(output[0], skip_special_tokens=True)
        return {"output": response}


In [5]:
from genai_stack.model import HuggingFaceModel
from genai_stack.stack.stack import Stack

# Override the model by passing the model_name as a dictionary to from_kwargs()
hugging_face_model = HuggingFaceModel.from_kwargs(model_name={"model_name": "meta-llama/Llama-2-13b-chat-hf"})
Stack(model=hugging_face_model)  # Initialize stack
model_response = hugging_face_model.predict("How many countries are there in the world?")
print(model_response["output"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


: 